In [7]:
import pandas as pd

# Load the dataset
file_path = "E:\Projects\Capstone\Final\Data\masterdatasubset.csv"  # Replace with your dataset path
data = pd.read_csv(file_path)

# Specify the columns to include
exclusive_labels = ['IAT', 'Tot sum', 'Tot size', 'Max', 'Header_Length', 'AVG', 'Magnitue', 
                    'Min', 'rst_count', 'Protocol Type', 'flow_duration', 'Std', 'Radius', 
                    'Variance', 'urg_count', 'Covariance', 'syn_count', 'Number', 'Weight', 'label']

# Filter the dataset to include only the specified columns
filtered_data = data[exclusive_labels]

<>:4: SyntaxWarning: invalid escape sequence '\P'
<>:4: SyntaxWarning: invalid escape sequence '\P'
C:\Users\anubh\AppData\Local\Temp\ipykernel_44296\3467048448.py:4: SyntaxWarning: invalid escape sequence '\P'
  file_path = "E:\Projects\Capstone\Final\Data\masterdatasubset.csv"  # Replace with your dataset path


In [9]:
n_samples = 10000

# Group by the 'label' column and sample
sampled_data = filtered_data.groupby('label', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))

# Reset index for the sampled dataset
sampled_data = sampled_data.reset_index(drop=True)

C:\Users\anubh\AppData\Local\Temp\ipykernel_44296\2981163042.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data = filtered_data.groupby('label', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))


In [10]:
sampled_data

,IAT,Tot sum,Tot size,Max,Header_Length,AVG,Magnitue,Min,rst_count,Protocol Type,flow_duration,Std,Radius,Variance,urg_count,Covariance,syn_count,Number,Weight,label
0,9.363198e-03,390.0,75.0,93.0,2769739.5,69.025000,11.742996,66.0,2784.3,6.0,51.478091,8.504897,12.027741,0.3,147.0,241.493750,0.0,5.5,38.5,Benign
1,1.665207e+08,18560.4,681.3,2962.0,14678.5,1218.720765,49.262322,75.0,11.0,7.1,0.040350,754.748399,1069.414581,1.0,0.1,573780.391586,0.0,13.5,244.6,Benign
2,6.743717e-03,3636.1,1167.1,1514.0,3356210.5,460.193810,26.795261,66.0,3109.6,6.0,41.180616,557.306340,788.150184,0.6,118.3,560928.870188,0.0,5.5,38.5,Benign
3,3.800821e-03,510.2,83.1,164.0,86236.2,102.032024,14.210188,68.6,99.6,8.2,1.949536,39.142768,55.356233,0.9,9.0,1709.493058,0.6,5.5,38.5,Benign
4,1.665232e+08,1404.8,150.6,265.6,27683.3,88.098932,13.229239,66.0,165.6,6.0,8.984087,48.411474,68.863610,1.0,79.0,4263.571035,1.8,13.5,244.6,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,1.676394e+08,2472.4,141.0,230.0,1361.9,160.772344,17.920271,56.4,4.2,9.3,29.059046,75.261171,106.689614,1.0,2.6,5695.307911,0.0,13.5,244.6,Web
79996,1.629801e-02,4616.6,652.4,1365.6,9988.1,815.081270,39.125976,66.0,26.3,8.2,545.685061,589.695054,833.954743,0.9,15.6,388226.858397,1.4,5.5,38.5,Web
79997,2.014730e-02,478.6,76.2,176.0,2845.8,99.925317,14.009528,74.6,20.7,9.3,127.597889,37.233245,52.655760,0.9,12.8,1568.530664,1.1,5.5,38.5,Web
79998,1.442299e-02,307.1,57.7,64.8,36483.3,55.635992,10.547503,47.3,82.2,14.8,229.822587,6.039241,8.540777,0.9,38.7,43.403429,0.0,5.5,38.5,Web


In [11]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
X = sampled_data.drop(columns=['label'])
y = sampled_data['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define base models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
svm = SVC(probability=True, random_state=42)

# Define meta-model
meta_model = LogisticRegression()

In [13]:
stack = StackingClassifier(
    estimators=[
        ('random_forest', rf),
        ('xgboost', xgb),
        ('svm', svm)
    ],
    final_estimator=meta_model,
    cv=5
)

# Train the stacking model
stack.fit(X_train, y_train)

# Make predictions
y_pred = stack.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Stacked Model: {accuracy:.2f}")

c:\Users\anubh\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:36:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


: 